In [ ]:
import os
import cv2
import albumentations as A

GS_FOLDER = r"C:\BPA\data\Gunshot"   #
IS_FOLDER = r"C:\BPA\data\Impact"    

GS_PREFIX = "GS - "            
IS_PREFIX = "IS - "             

GS_START_INDEX = 92             
IS_START_INDEX = 62            

AUG_PER_IMAGE = 4

MAX_SIZE = 10000    


augment = A.Compose([
    A.Rotate(limit=20, p=0.8),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.4),
    A.GaussNoise(p=0.4),
    A.MotionBlur(blur_limit=3, p=0.3),
])


def list_images(folder):
    exts = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")
    return [f for f in os.listdir(folder) if f.lower().endswith(exts)]


def safe_resize(image, max_size=MAX_SIZE):
    """Downscale very large images so OpenCV transforms don't crash."""
    h, w = image.shape[:2]
    if max(h, w) <= max_size:
        return image  # already safe
    scale = max_size / float(max(h, w))
    new_w = int(w * scale)
    new_h = int(h * scale)
    resized = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_AREA)
    return resized


def augment_folder(input_folder, prefix, start_index):
    """
    Read every image in input_folder,
    create AUG_PER_IMAGE augmented versions,
    and save as:  prefix + number  (e.g.  GS - 68.jpg, GS - 69.jpg, ...)
    """
    images = list_images(input_folder)
    images.sort()  # just for consistent order

    if not images:
        print(f"[WARN] No images found in: {input_folder}")
        return

    counter = start_index

    print(f"\n[INFO] Processing folder: {input_folder}")
    print(f"      Saving as: {prefix}<number>.jpg starting from {counter}")
    print(f"      Originals found: {len(images)}")

    for filename in images:
        img_path = os.path.join(input_folder, filename)

        try:
            image = cv2.imread(img_path)
        except cv2.error as e:
            print(f"  [ERR] OpenCV could not read '{img_path}': {e}")
            print("        Skipping this file.")
            continue

        if image is None:
            print(f"  [SKIP] '{img_path}' could not be loaded (None).")
            continue
        image = safe_resize(image, MAX_SIZE)

        for _ in range(AUG_PER_IMAGE):
            try:
                aug = augment(image=image)["image"]
            except Exception as e:
                print(f"  [ERR] Failed to augment '{filename}': {e}")
                break

            out_name = f"{prefix}{counter}.jpg"
            out_path = os.path.join(input_folder, out_name)
            cv2.imwrite(out_path, aug)
            print(f"  [SAVE] {out_name}")
            counter += 1

    print(f"[DONE] Folder {input_folder} → last index used: {counter - 1}")



def main():

    augment_folder(GS_FOLDER, GS_PREFIX, GS_START_INDEX)
    augment_folder(IS_FOLDER, IS_PREFIX, IS_START_INDEX)


if __name__ == "__main__":
    main()



[INFO] Processing folder: C:\BPA\data\Gunshot
      Saving as: GS - <number>.jpg starting from 92
      Originals found: 88
  [SAVE] GS - 92.jpg
  [SAVE] GS - 93.jpg
  [SAVE] GS - 94.jpg
  [SAVE] GS - 95.jpg
  [SAVE] GS - 96.jpg
  [SAVE] GS - 97.jpg
  [SAVE] GS - 98.jpg
  [SAVE] GS - 99.jpg
  [SAVE] GS - 100.jpg
  [SAVE] GS - 101.jpg
  [SAVE] GS - 102.jpg
  [SAVE] GS - 103.jpg
  [SAVE] GS - 104.jpg
  [SAVE] GS - 105.jpg
  [SAVE] GS - 106.jpg
  [SAVE] GS - 107.jpg
  [SAVE] GS - 108.jpg
  [SAVE] GS - 109.jpg
  [SAVE] GS - 110.jpg
  [SAVE] GS - 111.jpg
  [SAVE] GS - 112.jpg
  [SAVE] GS - 113.jpg
  [SAVE] GS - 114.jpg
  [SAVE] GS - 115.jpg
  [ERR] OpenCV could not read 'C:\BPA\data\Gunshot\GS-15.jpg': OpenCV(4.12.0) C:\miniconda3\conda-bld\opencv-suite_1761815220365\work\modules\imgcodecs\src\loadsave.cpp:79: error: (-215:Assertion failed) pixels <= CV_IO_MAX_IMAGE_PIXELS in function 'cv::validateInputImageSize'

        Skipping this file.
  [SAVE] GS - 116.jpg
  [SAVE] GS - 117.jpg
  [S